In [1]:
import torch
print("PyTorch 버전:", torch.__version__)
print("CUDA 사용 가능:", torch.cuda.is_available())

PyTorch 버전: 2.6.0+cu124
CUDA 사용 가능: True


In [2]:
import pandas as pd
import numpy as np


# 데이터 로드 및 전처리
train = pd.read_csv('./data/ratings_train.txt', sep='\t').dropna(subset=['document'])

# S-KoBERT 모델 불러오기
# jhgan/ko-sbert-multitask은 대표적인 S-KoBERT 모델
# 두 문장의 코사인 유사도를 조절함으로써 모델 학습
# 문장 길이와는 상관없이 문장의 의미가 중요. 따라서 L2나 내적이 아닌 코사인유사도를 사용
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('jhgan/ko-sbert-multitask', device='cuda')


# 임베딩 실행
# 모델의 입력을 list 형태로 받아야 안정적
# S-KoBERT 모델의 임베딩 과정
# 1. 15만개의 문장을 batch_size 크기로 분할하여 진행
# 2. 토큰화 : ['영화', '##가', '재미', '##있다']처럼, 의미 단위로 단어를 분리, 이 분리된 4개의 토큰들을 미리 만들어 둔 단어사전(vocab)을 보고 숫자로 변환
# 3. 계산 : Self-Attention, 서로가 서로를 참고하여 문맥속에서의 의미를 파악, 결과적으로 각 토큰별로 768차원의 4개 벡터로 변환 (4, 768)
# 4. 풀링 : 문장별로 토큰의 개수가 다르기때문에, 각 토큰들을 더하고 토큰의 개수로 나누어 (1,768)로 벡터를 요약.
print("Train 데이터 임베딩 중...")
train_embeddings = model.encode(train['document'].tolist(), show_progress_bar= True, batch_size=64)

# 출력 데이터 정보 확인
print(f"Train 임베딩 형태: {train_embeddings.shape}")
print("첫 번째 문장의 벡터값(10차원만):\n", train_embeddings[0][:10])

# 임베딩 결과 저장
np.save('./embeddings/train_embeddings.npy', train_embeddings)
print("임베딩 결과 저장 완료!")

C:\Users\xorua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Train 데이터 임베딩 중...


Batches: 100%|██████████| 2344/2344 [02:14<00:00, 17.43it/s]


Train 임베딩 형태: (149995, 768)
첫 번째 문장의 벡터값(10차원만):
 [ 0.83763146 -0.836729    0.05637764  0.5392365   0.48127806  0.20221631
  0.24015087  0.43785825 -0.92694783 -0.26269993]
임베딩 결과 저장 완료!


In [3]:
import faiss
import numpy as np
import pandas as pd

# 데이터 및 임베딩 로드
train = pd.read_csv('./data/ratings_train.txt', sep='\t').dropna(subset=['document']).reset_index(drop=True)
train_embeddings = np.load('./embeddings/train_embeddings.npy')

if len(train) != train_embeddings.shape[0]:
    print("경고: 데이터프레임과 임베딩의 개수가 맞지 않습니다.")

# Faiss 인덱스 생성 함수 정의
def create_index(embeddings, method='HNSWFLAT'):
    d = embeddings.shape[1]  # 벡터 차원

    if method == 'HNSWFLAT':
        index = faiss.IndexHNSWFlat(d, 32)
        index.add(embeddings)

    elif method == 'IVFFLAT':
        nlist = 100  # 클러스터 개수
        quantizer = faiss.IndexFlatL2(d)
        index = faiss.IndexIVFFlat(quantizer, d, nlist)
        index.train(embeddings)
        index.add(embeddings)

    elif method == 'IVFPQ':
        nlist = 100
        m = 8  # 서브 퀀타이저 개수
        quantizer = faiss.IndexFlatL2(d)
        index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)
        index.train(embeddings)
        index.add(embeddings)

    else:
        raise ValueError("Unknown method")

    return index

# 검색 함수 정의
def search_similar_reviews(query, model, index, k=5):
    # 쿼리 문장을 벡터로 변환
    query_vector = model.encode([query])

    # Faiss 검색
    distances, indices = index.search(query_vector, k)

    print(f"\n[검색 문장]: {query}")
    print("-" * 50)

    for i in range(k):
        idx = indices[0][i]
        dist = distances[0][i]
        review_text = train.iloc[idx]['document']
        print(f"{i+1}위 (거리: {dist:.4f}): {review_text}")

# ---------------------------------------------------------
# 실행 예시
# ---------------------------------------------------------

# 원하는 인덱스 방식 선택 (HNSWFLAT, IVFFLAT, IVFPQ)
method = 'HNSWFLAT'
index = create_index(train_embeddings, method=method)
print(f"{method} 인덱스 생성 완료!")

# 테스트
test_query = "스토리가 너무 감동적이고 배우들 연기가 최고였어요"
search_similar_reviews(test_query, model, index, k=5)

test_query2 = "시간 아깝다 진짜 재미없네"
search_similar_reviews(test_query2, model, index, k=5)

HNSWFLAT 인덱스 생성 완료!

[검색 문장]: 스토리가 너무 감동적이고 배우들 연기가 최고였어요
--------------------------------------------------
1위 (거리: 75.8030): 정말이지 스토리와 배우들의 연기 연출으로 엄청난 감동을 받았다.
2위 (거리: 80.6873): 너무나 감동적이었어요..! 연기 짱! 스토리도 짱 결말도 짱! 배우분들 스텝분들 수고하셨습니다
3위 (거리: 97.0027): 이 영화의 스토리와 인물들의 연기는 정말 최고다.
4위 (거리: 97.8980): 배우들 연기 모두 너무 좋았고 몰입도 최고..추천!
5위 (거리: 103.3614): 정말 재밌게 봤습니다. 스토리 좋고, 배우들 연기도 좋았어요!

[검색 문장]: 시간 아깝다 진짜 재미없네
--------------------------------------------------
1위 (거리: 61.1680): 아..너무 재미없다 정말 시간이 아깝다
2위 (거리: 76.7492): 진짜진짜 재미없음시간낭비
3위 (거리: 84.7812): 진짜 재미없어요~시간아깝습니다
4위 (거리: 85.7160): 드럽게 재미없다..시간이 아깝다
5위 (거리: 86.1421): 더럽게 재미없음ㅡㅡ 시간낭비임
